In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import Subset

In [2]:
batch_size = 32

# convert data to torch.FloatTensor
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, ), (0.5,))])

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

len(train_data), len(test_data)

(60000, 10000)

In [3]:
# number of teachers to essemble
num_teachers = 100

def get_data_loaders(train_data, num_teachers = 10):
    teacher_loaders = []
    data_size = len(train_data) // num_teachers

    for i in range(num_teachers):
        indices = list(range(i*data_size, (i+1) *data_size))
        subset_data = Subset(train_data, indices)
        loader = torch.utils.data.DataLoader(subset_data, batch_size=batch_size)
        teacher_loaders.append(loader)

    return teacher_loaders

teacher_loaders = get_data_loaders(train_data, num_teachers)
len(teacher_loaders)

100

In [4]:
student_train_data = Subset(test_data, list(range(9000)))
student_test_data = Subset(test_data, list(range(9000, 10000)))

student_train_loader = torch.utils.data.DataLoader(student_train_data, batch_size=batch_size)
student_test_loader = torch.utils.data.DataLoader(student_test_data, batch_size=batch_size)

len(student_train_loader), len(student_test_loader)

(282, 32)

In [9]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train(model, trainloader, criterion, optimizer, epochs=10, print_every=120):
    model.to(device)
    steps = 0
    running_loss = 0
    for e in range(epochs):
        # Model in training mode, dropout is on
        model.train()
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            steps += 1
            
            optimizer.zero_grad()
            
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

In [7]:
def predict(model, dataloader):
    outputs = torch.zeros(0, dtype=torch.long).to(device)
    model.to(device)
    model.eval()
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        output = model.forward(images)
        ps = torch.argmax(torch.exp(output), dim=1)
        outputs = torch.cat((outputs, ps))
    
    return outputs

In [8]:
# Instantiate and train the models for each teacher
def train_models(num_teachers):
    models = []
    for t in range(num_teachers):
        print("Training teacher {}".format(t+1))
        model = Network()
        criterion = nn.NLLLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.003)
        train(model, teacher_loaders[t], criterion, optimizer)
        models.append(model)
    return models

models = train_models(num_teachers)

Training teacher 1
Training teacher 2
Training teacher 3
Training teacher 4
Training teacher 5
Training teacher 6
Training teacher 7
Training teacher 8
Training teacher 9
Training teacher 10
Training teacher 11
Training teacher 12
Training teacher 13
Training teacher 14
Training teacher 15
Training teacher 16
Training teacher 17
Training teacher 18
Training teacher 19
Training teacher 20
Training teacher 21
Training teacher 22
Training teacher 23
Training teacher 24
Training teacher 25
Training teacher 26
Training teacher 27
Training teacher 28
Training teacher 29
Training teacher 30
Training teacher 31
Training teacher 32
Training teacher 33
Training teacher 34
Training teacher 35
Training teacher 36
Training teacher 37
Training teacher 38
Training teacher 39
Training teacher 40
Training teacher 41
Training teacher 42
Training teacher 43
Training teacher 44
Training teacher 45
Training teacher 46
Training teacher 47
Training teacher 48
Training teacher 49
Training teacher 50
Training 

In [34]:
import numpy as np

epsilon = 0.2

def aggregated_teacher(models, data_loader, epsilon):
    preds = torch.torch.zeros((len(models), 9000), dtype=torch.long)
    for i, model in enumerate(models):
        results = predict(model, data_loader)
        preds[i] = results
        
    labels = np.array([]).astype(int)
    for image_preds in np.transpose(preds):
        label_counts = np.bincount(image_preds, minlength=10)
        beta = 1 / epsilon

        for i in range(len(label_counts)):
            label_counts[i] += np.random.laplace(0, beta, 1)

        new_label = np.argmax(label_counts)
        labels = np.append(labels, new_label)
    
    return preds.numpy(), labels

In [35]:
teacher_models = models
preds, student_labels = aggregated_teacher(teacher_models, student_train_loader, epsilon)

In [37]:
preds.shape, student_labels

((100, 9000), array([7, 2, 1, ..., 6, 9, 0]))

In [38]:
from syft.frameworks.torch.differential_privacy import pate

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=student_labels, noise_eps=epsilon, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 1451.5129254649705
Data Dependent Epsilon: 6.689155555034445


In [39]:
def student_loader(student_train_loader, labels):
    for i, (data, _) in enumerate(iter(student_train_loader)):
        yield data, torch.from_numpy(labels[i*len(data):(i+1)*len(data)])

In [40]:
student_model = Network()
criterion = nn.NLLLoss()
optimizer = optim.Adam(student_model.parameters(), lr=0.001)
epochs = 10
student_model.to(device)
steps = 0
running_loss = 0
for e in range(epochs):
    # Model in training mode, dropout is on
    student_model.train()
    train_loader = student_loader(student_train_loader, student_labels)
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        steps += 1

        optimizer.zero_grad()
        output = student_model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if steps % 50 == 0:
            test_loss = 0
            accuracy = 0
            student_model.eval()
            with torch.no_grad():
                for images, labels in student_test_loader:
                    images, labels = images.to(device), labels.to(device)
                    log_ps = student_model(images)
                    test_loss += criterion(log_ps, labels).item()
                    
                    # Accuracy
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
            student_model.train()
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/len(student_train_loader)),
                  "Test Loss: {:.3f}.. ".format(test_loss/len(student_test_loader)),
                  "Test Accuracy: {:.3f}".format(accuracy/len(student_test_loader)))
            running_loss = 0

Epoch: 1/10..  Training Loss: 0.398..  Test Loss: 2.020..  Test Accuracy: 0.349
Epoch: 1/10..  Training Loss: 0.276..  Test Loss: 0.750..  Test Accuracy: 0.829
Epoch: 1/10..  Training Loss: 0.159..  Test Loss: 0.455..  Test Accuracy: 0.879
Epoch: 1/10..  Training Loss: 0.105..  Test Loss: 0.368..  Test Accuracy: 0.896
Epoch: 1/10..  Training Loss: 0.099..  Test Loss: 0.275..  Test Accuracy: 0.928
Epoch: 2/10..  Training Loss: 0.120..  Test Loss: 0.325..  Test Accuracy: 0.920
Epoch: 2/10..  Training Loss: 0.098..  Test Loss: 0.264..  Test Accuracy: 0.927
Epoch: 2/10..  Training Loss: 0.086..  Test Loss: 0.231..  Test Accuracy: 0.937
Epoch: 2/10..  Training Loss: 0.075..  Test Loss: 0.218..  Test Accuracy: 0.937
Epoch: 2/10..  Training Loss: 0.064..  Test Loss: 0.210..  Test Accuracy: 0.942
Epoch: 2/10..  Training Loss: 0.060..  Test Loss: 0.202..  Test Accuracy: 0.939
Epoch: 3/10..  Training Loss: 0.089..  Test Loss: 0.203..  Test Accuracy: 0.944
Epoch: 3/10..  Training Loss: 0.073..  T

In [41]:
t1_model = models[99]
t1_model.eval()
with torch.no_grad():
    test_loss = 0
    accuracy = 0
    for images, labels in student_test_loader:
        images, labels = images.to(device), labels.to(device)
        log_ps = t1_model(images)
        test_loss += criterion(log_ps, labels).item()

        # Accuracy
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))
    t1_model.train()
    print("Test Loss: {:.3f}.. ".format(test_loss),
          "Test Accuracy: {:.3f}".format(accuracy))

Test Loss: 11.291..  Test Accuracy: 28.812
